<a href="https://colab.research.google.com/github/abood-W/ai_python/blob/main/group_of_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mistralai
!pip install langchain langchain-mistralai faiss-cpu


In [2]:
!pip install langchain langchain-mistralai faiss-cpu
!pip install langchain_community
!pip install -U langchain-tavily
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 

In [3]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=e0ed5c1769e31f0bdd48cc585d6e35d234b0d223a98283f6861cedf48cdc954d
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [4]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, Tool
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from google.colab import userdata
import csv
import os

In [5]:
texts = [
    "Newton's second law states that Force equals Mass times Acceleration (F=ma).",
    "LangChain is a framework for building context-aware reasoning applications.",
    "Python was created by Guido van Rossum and first released in 1991."
]

In [6]:
def critic(task: str , result:str , success:bool):
    log_file = "agent_feedback.csv"
    file_exists = os.path.isfile(log_file)

    with open(log_file, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["task", "result", "success"])
        writer.writerow([task, result, int(success)])
    return "Feedback recorded"

In [7]:
api_key = userdata.get('MISTRAL_API_KEY')
llm = ChatMistralAI(model="mistral-small",
                    temperature=0,
                    api_key=api_key
                    )

In [8]:
embeddings = MistralAIEmbeddings(api_key=api_key)

vectorstore = FAISS.from_texts(texts, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_mistralai/embeddings.py:186: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [9]:
def math_tool(expr: str) -> str:
    return str(eval(expr))

In [11]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
wiki_api = WikipediaAPIWrapper()
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)

**MEMORY**

In [12]:
from langchain.memory import ConversationBufferMemory

In [13]:
math_tools = [Tool(name="MathTool", func=lambda x: str(eval(x)), description="Do math calculations")]
math_agent = initialize_agent(
    math_tools,
    llm,
    agent="conversational-react-description", memory=ConversationBufferMemory(return_messages=True), verbose=True
    )

/tmp/ipython-input-953579063.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  agent="conversational-react-description", memory=ConversationBufferMemory(return_messages=True), verbose=True
/tmp/ipython-input-953579063.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  math_agent = initialize_agent(


In [14]:
wiki_agent = initialize_agent(
    [wiki_tool],
    llm,
    agent="conversational-react-description",
    memory=ConversationBufferMemory(return_messages=True), verbose=True)


In [15]:
from langchain.chains import RetrievalQA

In [16]:
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
rag_agent = initialize_agent([Tool(name="RAGTool", func=rag_chain.run, description="Retrieve and answer from documents")],
                             llm, agent="conversational-react-description", memory=ConversationBufferMemory(return_messages=True), verbose=True)

In [17]:
tools = [
    Tool(name="MathExpert", func=math_agent.run, description="Expert in math calculations"),
    Tool(name="WikiExpert", func=wiki_agent.run, description="Eaxpert in Wikipedia knowledge"),
    Tool(name="RAGExpert", func=rag_agent.run, description="Expert in retrieving from custom documents"),
    Tool(name="Critic", func=lambda x: critic(x["task"], x["result"], x["success"]), description="Evaluate and log results")
]


In [18]:
main_memory = ConversationBufferMemory(memory_key="chat_history", input_key="input", return_messages=True)

In [19]:
from langchain.agents import initialize_agent

main_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="conversational-react-description",
    memory=main_memory,
    verbose=True
)

In [20]:
response = main_agent.invoke({"input": "What's the capital of France?"})
print(response["output"])




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No.
AI: The capital of France is Paris.

> Finished chain.
The capital of France is Paris.


In [21]:
response = main_agent.invoke({"input": "What's the square root of 25?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: MathExpert
Action Input: sqrt(25)
Observation: The square root of 25 is 5.

AI: The square root of 25 is 5.

> Finished chain.
The square root of 25 is 5.


In [22]:
response = main_agent.invoke({"input": "Search about relativity theorem"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: WikiExpert
Action Input: Relativity theorem
Observation: The theory of relativity is a scientific theory proposed by Albert Einstein that describes the laws of physics in the presence of gravity. The theory is divided into two parts, special relativity and general relativity. Special relativity applies to all physical phenomena in the absence of gravity. It introduces two postulates: first, that the laws of physics are the same in all inertial frames of reference; and second, that the speed of light in a vacuum is the same for all observers, regardless of their motion or the motion of the source of the light. General relativity, on the other hand, is a theory of gravitation that was developed by Einstein between 1907 and 1915. According to general relativity, the observed gravitational effect between masses results from their warping of spacetime.

Thought: Do I need to use a tool? No
AI: The theory o

In [23]:
response = main_agent.invoke({"input": "may name is mohammed"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello Mohammed, it's nice to meet you! Is there something specific you would like to ask or discuss? I'm here to help with a wide range of topics, from answering questions to providing explanations and descriptions on a variety of subjects.

> Finished chain.
Hello Mohammed, it's nice to meet you! Is there something specific you would like to ask or discuss? I'm here to help with a wide range of topics, from answering questions to providing explanations and descriptions on a variety of subjects.


In [24]:
response = main_agent.invoke({"input": "what is may name"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: The user seems to have provided their name, but they have misspelled "my" as "may". I do not need to use a tool for this response.

AI: Hello Mohammed, it's nice to meet you! I apologize for the misunderstanding in my previous response. I'm here to help with a wide range of topics, from answering questions to providing explanations and descriptions on a variety of subjects. Is there something specific you would like to ask or discuss?

> Finished chain.
Hello Mohammed, it's nice to meet you! I apologize for the misunderstanding in my previous response. I'm here to help with a wide range of topics, from answering questions to providing explanations and descriptions on a variety of subjects. Is there something specific you would like to ask or discuss?


In [25]:
ans = main_agent.invoke("what is may name")
print(ans['output'])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello Mohammed, it's nice to meet you! I apologize for the misunderstanding in my previous response. I'm here to help with a wide range of topics, from answering questions to providing explanations and descriptions on a variety of subjects. Your name is Mohammed, as you mentioned earlier. Is there something specific you would like to ask or discuss?

> Finished chain.
Hello Mohammed, it's nice to meet you! I apologize for the misunderstanding in my previous response. I'm here to help with a wide range of topics, from answering questions to providing explanations and descriptions on a variety of subjects. Your name is Mohammed, as you mentioned earlier. Is there something specific you would like to ask or discuss?


In [29]:
"""import pandas as pd
df = pd.read_csv("agent_feedback.csv")
print(df)"""

'import pandas as pd\ndf = pd.read_csv("agent_feedback.csv")\nprint(df)'

In [31]:
!pip install langchain_community
!pip install sentence-transformers

In [32]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [33]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**save it to Long term memory**

In [34]:
def save_experience(task, result, feedback):
    text = f"Task: {task}\nResult: {result}\nFeedback: {feedback}"
    vectorstore.add_texts([text])
    print("Experience saved to long-term memory.")

**search in long term memory and printing it **

In [35]:
def recall_experience(query, k=2):
    docs = vectorstore.similarity_search(query, k=k)
    if docs:
        print("Retrieved past experiences:")
        for d in docs:
            print(d.page_content)
    return docs


**to know if the answer is correct or no based on LLM ddesition**

In [48]:
def evaluate_with_llm(question:str , answer:str , correct_answer:str =None) ->bool:
    eval_prompt = f"""
    Question: {question}
    Answer: {answer}
    """

    eval_prompt = f"""
    Question: {question}
    Answer: {answer}
    """

    if correct_answer:
        eval_prompt += f"\nExpected Correct Answer: {correct_answer}"

    eval_prompt += """
    Task: Please evaluate if the Answer is correct given the Question.
    Reply strictly with 'Yes' if correct, 'No' if not.
    """

    evaluation = llm.invoke(eval_prompt)

    result_text = evaluation.content.strip().lower()
    return "yes" in result_text

auto routing with Long term memory


In [49]:
def ask_with_longterm_memory(agent, query, correct_answer=None):

    past_docs = recall_experience(query)
    context = "\n".join([d.page_content for d in past_docs]) if past_docs else ""


    final_query = f"Context:\n{context}\n\nNew Question: {query}" if context else query


    result = agent.run(final_query)


    #success = any(str(num) in result for num in range(0, 10000))
    success = evaluate_with_llm(query, result, correct_answer)

    feedback = critic(task=query, result=result, success=success)


    save_experience(query, result, feedback)

    return {"answer": result, "feedback": feedback}


In [37]:
output = ask_with_longterm_memory(main_agent, "Calculate 12 * 15")
print(output)

/tmp/ipython-input-3481225336.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(final_query)


Retrieved past experiences:
Newton's second law states that Force equals Mass times Acceleration (F=ma).
LangChain is a framework for building context-aware reasoning applications.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: The product of 12 and 15 is 180.

> Finished chain.
Experience saved to long-term memory.
{'answer': 'The product of 12 and 15 is 180.', 'feedback': 'Feedback recorded'}


In [38]:
output = ask_with_longterm_memory(main_agent, "Calculate 2 * 15")
print(output)

Retrieved past experiences:
Task: Calculate 12 * 15
Result: The product of 12 and 15 is 180.
Feedback: Feedback recorded
Newton's second law states that Force equals Mass times Acceleration (F=ma).


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: The product of 2 and 15 is 30.

> Finished chain.
Experience saved to long-term memory.
{'answer': 'The product of 2 and 15 is 30.', 'feedback': 'Feedback recorded'}


In [39]:
output = ask_with_longterm_memory(main_agent, "Retrieve from Wikipedia the page about '2020 phones'")
print(output)

Retrieved past experiences:
Task: Calculate 12 * 15
Result: The product of 12 and 15 is 180.
Feedback: Feedback recorded
Task: Calculate 2 * 15
Result: The product of 2 and 15 is 30.
Feedback: Feedback recorded


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: WikiExpert
Action Input: '2020 phones'
Observation: The page about '2020 phones' on Wikipedia provides information about the smartphones that were released in the year 2020. Some of the notable phones released in 2020 include the Samsung Galaxy S20 series, the Apple iPhone 12 series, and the Google Pixel 5. The page provides detailed specifications, features, and release dates for each of these phones, as well as other phones released in 2020.

AI: The page about '2020 phones' on Wikipedia provides information about the smartphones that were released in the year 2020. Some of the notable phones released in 2020 include the Samsung Galaxy S20 series, the Apple iPhone 12 series, and the Google Pi

In [51]:
output = ask_with_longterm_memory(main_agent, "what is my name ?")
print(output)

Retrieved past experiences:
Task: what is my name ?
Result: Mohammed, as you mentioned earlier, it's nice to interact with you again! Is there something specific you would like to ask or discuss?
Feedback: Feedback recorded
Python was created by Guido van Rossum and first released in 1991.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Mohammed, as we have communicated before, it's great to interact with you again! Is there something specific you would like to ask or discuss? I'm here to help with a wide range of topics, from answering questions to providing explanations and descriptions on a variety of subjects. Regarding your question, your name is Mohammed, as you mentioned earlier. Is there something else you would like to know or discuss?

> Finished chain.
Experience saved to long-term memory.
{'answer': "Mohammed, as we have communicated before, it's great to interact with you again! Is there something specific you would like to ask or discuss? 

In [47]:
output = ask_with_longterm_memory(main_agent, "What is newton second law?", correct_answer="F = ma")
print(output)


Retrieved past experiences:
Newton's second law states that Force equals Mass times Acceleration (F=ma).
Task: Calculate 2 * 15
Result: The product of 2 and 15 is 30.
Feedback: Feedback recorded


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Newton's second law, also known as F=ma, states that the force acting on an object is equal to the mass of that object times its acceleration. In other words, Force (F) equals Mass (m) times Acceleration (a). This law helps us understand how objects move and how forces affect their motion.

> Finished chain.
Experience saved to long-term memory.
{'answer': "Newton's second law, also known as F=ma, states that the force acting on an object is equal to the mass of that object times its acceleration. In other words, Force (F) equals Mass (m) times Acceleration (a). This law helps us understand how objects move and how forces affect their motion.", 'feedback': 'Feedback recorded'}
